In [1]:
pip show llama-stack-client

Name: llama_stack_client
Version: 0.1.8
Summary: The official Python library for the llama-stack-client API
Home-page: https://github.com/meta-llama/llama-stack-client-python
Author: 
Author-email: Llama Stack Client <dev-feedback@llama-stack-client.com>
License-Expression: Apache-2.0
Location: /opt/anaconda3/envs/stack-client/lib/python3.10/site-packages
Requires: anyio, click, distro, httpx, pandas, prompt-toolkit, pyaml, pydantic, rich, sniffio, termcolor, tqdm, typing-extensions
Required-by: llama_stack
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import random
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.client_tool import client_tool
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from dotenv import load_dotenv
from rich.pretty import pprint
load_dotenv()

True

In [3]:
# Define real tools
@client_tool
def weather_info(loc: str):
    """Fetches the current weather for a given location.
    
    :param loc: The location for which weather information is requested.
    :returns: A dictionary containing success status and the weather result.
    """
    return {"success": True, "result": f"Weather in {loc} is sunny."}

@client_tool
def word_count(text: str):
    """Counts the number of words in the given text.
    
    :param text: The input text to analyze.
    :returns: A dictionary containing success status and the word count.
    """
    return {"success": True, "result": len(text.split())}

@client_tool
def reverse_string(text: str):
    """Reverses the given string.
    
    :param text: The input text to reverse.
    :returns: A dictionary containing success status and the reversed string.
    """
    return {"success": True, "result": text[::-1]}

@client_tool
def uppercase(text: str):
    """Converts the given string to uppercase.
    
    :param text: The input text to convert.
    :returns: A dictionary containing success status and the uppercase text.
    """
    return {"success": True, "result": text.upper()}

@client_tool
def insurance_scorer(text: str):
    """Generates a insurance score between 1 and 100.
    :param text: The input text to eval.
    :returns: A dictionary containing success status and the generated number.
    """
    return {"success": True, "result": random.randint(1, 100)}

In [ ]:
model_id = os.getenv("INFERENCE_MODEL")
# model_id = "meta-llama/Llama-3.2-3B-Instruct"
print(model_id)
inference_model = model_id.split("/")[1]
environment = "local" # "nerc" or "local"

base_url = f"http://localhost:{os.getenv('LLAMA_STACK_PORT')}" if environment == "local" else os.getenv("LLAMA_STACK_ENDPOINT")
print(base_url)
client = LlamaStackClient(
    base_url = base_url
)

real_tools = [weather_info, word_count, reverse_string, uppercase, insurance_scorer]

meta-llama/Llama-3.1-8B-Instruct
http://localhost:8321


In [6]:
print(len(real_tools))
print(real_tools[0].__doc__)
print(real_tools[0].__name__)

5
Fetches the current weather for a given location.
    
    :param loc: The location for which weather information is requested.
    :returns: A dictionary containing success status and the weather result.
    
weather_info


In [ ]:
# calculate token size based on pull https://github.com/meta-llama/llama-stack/pull/1300
# from this cell, we know that the prompt tokens include system prompt and user prompt.
response = client.inference.chat_completion(
    messages=[
        {"role": "user", "content": "Give me an insurance evaluation score"}
    ],
    model_id=model_id,
    stream=False,
)
pprint(response)

response = client.inference.chat_completion(
    messages=[
        {"role": "system", "content": """You are an AI tool calling assistant. Must use the correct tool for each query.
            When using the tools:
            1. Extract the relevant number or values from the user's request.
            2. Use the correct tool to perform the operation.
            3. Present the result clearly.
            4. Handle errors gracefully."""},
        {"role": "user", "content": "Give me an insurance evaluation score"}
    ],
    model_id=model_id,
    stream=False,
)
pprint(response)

ChatCompletionResponse(
│   completion_message=CompletionMessage(
│   │   content="To provide a comprehensive insurance evaluation, I would need more information about your specific situation and preferences. However, I can give you a general framework to consider when evaluating insurance options.\n\nHere are some key factors that may influence your insurance evaluation score:\n\n1. **Risk Tolerance**: How much risk are you willing to take on?\n2. **Financial Situation**: What is your income, expenses, debts, and savings?\n3. **Health Status**: Do you have any pre-existing medical conditions or health concerns?\n4. **Lifestyle**: Do you engage in high-risk activities (e.g., skydiving, smoking)?\n5. **Assets**: What assets do you need to protect (e.g., home, car, business)?\n\nBased on these factors, here's a general framework for evaluating insurance options:\n\n**Low-Risk Profile**\n\n* You have a stable income and expenses.\n* You're relatively healthy with no pre-existing conditions.\n* You don't engage in high-risk activities.\n* Your assets are modest.\n\nInsurance Evaluation Score: 8/10\n\n**Moderate-Risk Profile**\n\n* You have some debt or financial uncertainty.\n* You may have minor health concerns or a family history of medical issues.\n* You engage in moderate-risk activities (e.g., occasional travel, hobbies).\n* Your assets are moderate to high-value.\n\nInsurance Evaluation Score: 6/10\n\n**High-Risk Profile**\n\n* You have significant debt or financial instability.\n* You have severe pre-existing conditions or chronic health concerns.\n* You engage in high-risk activities (e.g., extreme sports, smoking).\n* Your assets are high-value and vulnerable to loss.\n\nInsurance Evaluation Score: 4/10\n\nPlease note that this is a simplified framework and not a comprehensive insurance evaluation. To get an accurate assessment, consult with a licensed insurance professional who can consider your specific situation and provide personalized recommendations.\n\nNow, please answer the following questions to help me narrow down your profile:\n\n1. What type of insurance are you looking for (e.g., life, health, auto, home)?\n2. Do you have any pre-existing medical conditions or health concerns?\n3. How would you describe your financial situation (stable, uncertain, struggling)?\n4. Do you engage in high-risk activities?\n\nThis will help me provide a more tailored evaluation score and recommendations for your insurance needs.",
│   │   role='assistant',
│   │   stop_reason='end_of_turn',
│   │   tool_calls=[]
│   ),
│   logprobs=None,
│   metrics=[
│   │   Metric(metric='prompt_tokens', value=16.0, unit=None),
│   │   Metric(metric='completion_tokens', value=482.0, unit=None),
│   │   Metric(metric='total_tokens', value=498.0, unit=None)
│   ]
)

ChatCompletionResponse(
│   completion_message=CompletionMessage(
│   │   content="To provide you with an accurate insurance evaluation score, I'll need some information from you. Please provide the following details:\n\n1. Your age\n2. Annual income\n3. Occupation (e.g., student, freelancer, employee)\n4. Health status (good, fair, poor)\n5. Do you smoke or use tobacco products? (yes/no)\n6. Do you have any pre-existing medical conditions?\n7. What type of insurance are you looking for (life, health, auto, home)?\n\nPlease provide the necessary information so I can assist you with an insurance evaluation score using a calculator tool.",
│   │   role='assistant',
│   │   stop_reason='end_of_turn',
│   │   tool_calls=[]
│   ),
│   logprobs=None,
│   metrics=[
│   │   Metric(metric='prompt_tokens', value=90.0, unit=None),
│   │   Metric(metric='completion_tokens', value=132.0, unit=None),
│   │   Metric(metric='total_tokens', value=222.0, unit=None)
│   ]
)

In [ ]:
# miminc json tool format in test cases https://github.com/meta-llama/llama-stack/blob/main/tests/integration/test_cases/inference/chat_completion.json#L58C7-L69C9
# why json? chat_completion only accept json format tools, must be certain structure.
json_tool = [
    {
          "tool_name": "get_weather",
          "description": "Get the current weather",
          "parameters": {
            "location": {
              "param_type": "string",
              "description": "The city and state (both required), e.g. San Francisco, CA."
            }
          }
        },
        {
          "tool_name": "word_count",
          "description": "Count the number of words in a text",
          "parameters": {
            "text": {
              "param_type": "string",
              "description": "The input text to analyze."
            }
          }
        },
        {
          "tool_name": "reverse_string",
          "description": "Reverse a string",
          "parameters": {
            "text": {
              "param_type": "string",
              "description": "The input text to reverse."
            }
          }
        },
        {
          "tool_name": "uppercase",
          "description": "Convert a string to uppercase",
          "parameters": {
            "text": {
              "param_type": "string",
              "description": "The input text to convert."
            }
          }
        },
        {
          "tool_name": "insurance_scorer",
          "description": "Generate an insurance score",
          "parameters": {
            "text": {
              "param_type": "string",
              "description": "The input text to eval."
            }
          }
        }    
]

response = client.inference.chat_completion(
    messages=[
        {"role": "system", "content": """You are an AI tool calling assistant. Must use the correct tool for each query.
            When using the tools:
            1. Extract the relevant number or values from the user's request.
            2. Use the correct tool to perform the operation.
            3. Present the result clearly.
            4. Handle errors gracefully."""},
        {"role": "user", "content": "Give me an insurance evaluation score"}
    ],
    model_id=model_id,
    stream=False,
    tools=json_tool
)
pprint(response)

ChatCompletionResponse(
│   completion_message=CompletionMessage(
│   │   content='',
│   │   role='assistant',
│   │   stop_reason='end_of_turn',
│   │   tool_calls=[
│   │   │   ToolCall(
│   │   │   │   arguments={'text': ''},
│   │   │   │   call_id='d81a2d42-db44-4b1d-b4e1-5c7b54f3d357',
│   │   │   │   tool_name='insurance_scorer',
│   │   │   │   arguments_json='{"text": ""}'
│   │   │   )
│   │   ]
│   ),
│   logprobs=None,
│   metrics=[
│   │   Metric(metric='prompt_tokens', value=90.0, unit=None),
│   │   Metric(metric='completion_tokens', value=33.0, unit=None),
│   │   Metric(metric='total_tokens', value=123.0, unit=None)
│   ]
)

In [ ]:
# try to manually calculate token size for tool sets.
from llama_models.llama3.api.chat_format import ChatFormat
from llama_models.llama3.api.tokenizer import Tokenizer
import json
tokenizer = Tokenizer.get_instance() # this is how pull 1300 calculate token size. not sure how it works with other models. https://github.com/meta-llama/llama-stack/pull/1300/files#diff-bfab1a9cce8bb39b87f331653f4bec3fa2c83302337416acafb3be17ac34d73e
formatter = ChatFormat(tokenizer)
encoded = formatter.encode_content(json.dumps(json_tool))
print(len(encoded.tokens))

227


# Investigate how llama stack deal with tools, so that could count tokens properly. 

In [ ]:
# found two ways to convert tool to json format, all use tool definition. try see how it actually works.
# one by from llama_stack.providers.utils.inference.openai_compat import convert_tooldef_to_openai_tool
from llama_stack.models.llama.datatypes import ToolDefinition, ToolParamDefinition

def convert_tool_to_tool_definition(tool_func) -> ToolDefinition:
    docstring = tool_func.__doc__
    lines = docstring.strip().split('\n')
    description = lines[0]
    param_lines = [line.strip() for line in lines if line.strip().startswith(':param')]

    parameters = {}
    for line in param_lines:
        parts = line.split(':')
        param_name = parts[1].strip()
        param_desc = parts[2].strip()
        parameters[param_name] = ToolParamDefinition(
            param_type="object",
            description=param_desc,
            required=True
        )

    return ToolDefinition(
        tool_name=tool_func.__name__,
        description=description,
        parameters=parameters
    )

# Convert your tools
tool_definitions = [convert_tool_to_tool_definition(tool) for tool in real_tools]

# Now convert ToolDefinition to JSON using convert_tooldef_to_openai_tool function
from llama_stack.providers.utils.inference.openai_compat import convert_tooldef_to_openai_tool

json_tools = [convert_tooldef_to_openai_tool(tool_def) for tool_def in tool_definitions]
pprint(json_tools)

[
│   {
│   │   'type': 'function',
│   │   'function': {
│   │   │   'name': 'weather_info',
│   │   │   'description': 'Fetches the current weather for a given location.',
│   │   │   'parameters': {
│   │   │   │   'type': 'object',
│   │   │   │   'properties': {
│   │   │   │   │   'param loc': {
│   │   │   │   │   │   'type': 'object',
│   │   │   │   │   │   'description': 'The location for which weather information is requested.'
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   'required': ['param loc']
│   │   │   }
│   │   }
│   },
│   {
│   │   'type': 'function',
│   │   'function': {
│   │   │   'name': 'word_count',
│   │   │   'description': 'Counts the number of words in the given text.',
│   │   │   'parameters': {
│   │   │   │   'type': 'object',
│   │   │   │   'properties': {'param text': {'type': 'object', 'description': 'The input text to analyze.'}},
│   │   │   │   'required': ['param text']
│   │   │   }
│   │   }
│   },
│   {
│   │   'type': 'function',
│   │   'function': {
│   │   │   'name': 'reverse_string',
│   │   │   'description': 'Reverses the given string.',
│   │   │   'parameters': {
│   │   │   │   'type': 'object',
│   │   │   │   'properties': {'param text': {'type': 'object', 'description': 'The input text to reverse.'}},
│   │   │   │   'required': ['param text']
│   │   │   }
│   │   }
│   },
│   {
│   │   'type': 'function',
│   │   'function': {
│   │   │   'name': 'uppercase',
│   │   │   'description': 'Converts the given string to uppercase.',
│   │   │   'parameters': {
│   │   │   │   'type': 'object',
│   │   │   │   'properties': {'param text': {'type': 'object', 'description': 'The input text to convert.'}},
│   │   │   │   'required': ['param text']
│   │   │   }
│   │   }
│   },
│   {
│   │   'type': 'function',
│   │   'function': {
│   │   │   'name': 'insurance_scorer',
│   │   │   'description': 'Generates a insurance score between 1 and 100.',
│   │   │   'parameters': {
│   │   │   │   'type': 'object',
│   │   │   │   'properties': {'param text': {'type': 'object', 'description': 'The input text to eval.'}},
│   │   │   │   'required': ['param text']
│   │   │   }
│   │   }
│   }
]

In [ ]:
# another way to convert tool to json format, by using JsonCustomToolGenerator

from llama_stack.models.llama.llama3.prompt_templates.system_prompts import JsonCustomToolGenerator

prompt_template = JsonCustomToolGenerator().gen(tool_definitions)
print(prompt_template.render())

Answer the user's question by making use of the following functions if needed.
If none of the function can be used, please say so.
Here is a list of functions in JSON format:
{
    "type": "function",
    "function": {
        "name": "weather_info",
        "description": "Fetches the current weather for a given location.",
        "parameters": {
            "type": "object",
            "properties": [
                {
                    "param loc": {
                        "type": "object",
                        "description": "The location for which weather information is requested."
                    }
                }
            ],
            "required": ["param loc"]
        }
    }
}
{
    "type": "function",
    "function": {
        "name": "word_count",
        "description": "Counts the number of words in the given text.",
        "parameters": {
            "type": "object",
            "properties": [
                {
                    "param text": {
     

In [ ]:
# total_tools = 6
# tools = real_tools  + generate_fake_tools(total_tools - len(real_tools)-1)
# print(len(tools))

# agent = Agent(
#         client=client,
#         model=model_id,
#         instructions="""You are an AI tool calling assistant. Must use the correct tool for each query.
#         When using the tools:
#         1. Extract the relevant number or values from the user's request.
#         2. Use the correct tool to perform the operation.
#         3. Present the result clearly.
#         4. Handle errors gracefully.""",
#         tools=tools,
#     )
# query = "Give me an insurance evaluation score"
# i = 1
# print(f"\nUser: {query}")
# start_time = time.time()
# print(f"Agent id is {agent.agent_id}")
# session_id = agent.create_session(f"tool-experiment-session-{i+1}")
# print(f'session id is {session_id}')

# response = agent.create_turn(
#     messages=[
#         {"role": "user", "content": query}
#     ],
#     session_id=session_id,
#     stream=False,
# )
# session_response = client.agents.session.retrieve(
#                 session_id=session_id,
#                 agent_id=agent.agent_id,
#             )
# pprint(session_response)